In [1]:
with open("raw_output.txt", "r", encoding="utf-8") as file:
    s = file.read()

print(s)

BATCH TEXT EXTRACTION STARTED
Total files to process: 152

FILE: mark-abenaki_page_001.png
PROCESSED: 1/152

# ST. MARK. CHAPTER I.

1\. U waji môjatak Sazos olôjmowôgan kchiniwaskw
wnamôna;

The beginning of the gospel of Jesus Christ, the Son of
God;

Commencement de la Bonne Nouvelle de Jésus Christ, le
Fils de Dieu;

2\. Akwôbawikhôzik nikôniwawôjmowinnowikoktali, Lôbi,
knikôni atkwai pjidakawlen noji kiwagimok, tônwaji kata
wlitokw akwôbôsaan.

As it is written in the prophets, Behold, I send my
messenger before thy face, which shall prepare thy way
before thee.

Il était écrit dans le livre du prophète Isaïe : Voici que
j'envoie mon messager devant toi, pour préparer la route.

3\. Pazgo pizwakamigok pmi li kôgôlwa, Kataiwi kistokw
akwôbosatji kzôgmômna, sasagitokw wdôwdial.

The voice of one crying in the wilderness, Prepare ye the
way of the Lord, make his paths straight.

A travers le désert, une voix crie : Préparez le chemin du
Seigneur,aplanissez sa route.

4\. Azô pmi sogn

In [55]:
import re

split_result = re.split(r'==+', s)

new_s = " ".join(split_result[1::2])
print(new_s)

In [62]:
with open("edited.txt", "r", encoding="utf-8") as file:
    s = file.read()

chapters = re.split(r'# CHAP\. [IVXLCDM]+\.', s)

chapters = [ch.strip() for ch in chapters[1:]]
print(len(chapters))



16


In [63]:
def split_chapter_to_verses(text):
    # Split before each number+dot+space, but only at the start of a verse number
    items = re.split(r'(?<!\d)(?=\d+\\\.\s)', text)

    # Remove any accidental empty strings
    items = [item.strip() for item in items if item.strip()]
    return items

In [64]:
split_chapter_to_verses(chapters[0])

['1\\. U waji môjatak Sazos olôjmowôgan kchiniwaskw\nwnamôna;\n\nThe beginning of the gospel of Jesus Christ, the Son of\nGod;\n\nCommencement de la Bonne Nouvelle de Jésus Christ, le\nFils de Dieu;',
 "2\\. Akwôbawikhôzik nikôniwawôjmowinnowikoktali, Lôbi,\nknikôni atkwai pjidakawlen noji kiwagimok, tônwaji kata\nwlitokw akwôbôsaan.\n\nAs it is written in the prophets, Behold, I send my\nmessenger before thy face, which shall prepare thy way\nbefore thee.\n\nIl était écrit dans le livre du prophète Isaïe : Voici que\nj'envoie mon messager devant toi, pour préparer la route.",
 '3\\. Pazgo pizwakamigok pmi li kôgôlwa, Kataiwi kistokw\nakwôbosatji kzôgmômna, sasagitokw wdôwdial.\n\nThe voice of one crying in the wilderness, Prepare ye the\nway of the Lord, make his paths straight.\n\nA travers le désert, une voix crie : Préparez le chemin du\nSeigneur,aplanissez sa route.',
 '4\\. Azô pmi sognebôlwassa pizwakamigok ta\nkinohomassossa ônalzowi sognebôzwôgan waji\nanhaldamôwadimek saagawa

In [68]:
def split_verse(text, chapter_number):
    # add chapter number to each verse?

    # split into number and text. parse number and add it to a tuple with chapter number
    items = re.split(r'\\\.\s+', text)
    verse_id = (chapter_number, int(items[0]))

    verse_text = " ".join(items[1:])
    translations = re.split(r'\n\n', verse_text)
    translations = [re.sub(r'\n', ' ', text) for text in translations]

    return verse_id, translations


In [69]:
# check to make sure that every verse of all chapters has exactly 3 translations
for i, chapter in enumerate(chapters):
    verses = split_chapter_to_verses(chapter)
    for j, verse in enumerate(verses):
        verse_id, translations = split_verse(verse, i + 1)
        if len(translations) != 3:
            print(f"Chapter {i + 1}, Verse {j + 1} has {len(translations)} translations: {translations}")

In [72]:
# create a pandas DataFrame with columns: chapter, verse, abenaki, english, french
import pandas as pd
data = []
for i, chapter in enumerate(chapters):
    verses = split_chapter_to_verses(chapter)
    for j, verse in enumerate(verses):
        verse_id, translations = split_verse(verse, i + 1)
        if len(translations) == 3:
            data.append({
                'chapter': verse_id[0],
                'verse': verse_id[1],
                'abenaki': translations[0].strip(),
                'english': translations[1].strip(),
                'french': translations[2].strip()
            })
df = pd.DataFrame(data)
df.to_csv("bible_translations.csv", index=False, encoding='utf-8')

In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def extract_meaningful_lemmas(text):
    """
    Extract lemmas of non-stop words from English text
    Returns a list of meaningful lexemes
    """
    # Process the text
    doc = nlp(text)
    
    # Extract lemmas of non-stop words, excluding punctuation and whitespace
    meaningful_lemmas = [
        token.lemma_.lower() 
        for token in doc 
        if not token.is_stop 
        and not token.is_punct 
        and not token.is_space
        and token.lemma_.strip()  # Exclude empty lemmas
    ]
    
    return meaningful_lemmas

# Example usage
sample_text = "The quick brown foxes are running through the beautiful forest."
lemmas = extract_meaningful_lemmas(sample_text)
print(f"Original text: {sample_text}")
print(f"Meaningful lemmas: {lemmas}")